# Context
`pyturbo`  library is provided by twiinIT to assembly a simple turbofan system.

The library is made of components: 

- `compressor` : fluid out is computed from fluid in and power provided by shaft with constant efficiency. 
- `combustor` : combustion is made considering constant FHV.
- `turbine` : power is extracted from fluid in considering a given expansion ratio and constant efficiency. 
- `ìnlet` and `nozzle`are computed `drag`and `thrust`from fluid conditions, ambiant pressure and throat section. 
- `nacelle`: envelop over the engine
There are also `ogv`, `intermediate_casing` and `trf` . 

They are numerical components:

- `fluid_spitter`is used to manage fluid splitter (to split the flow into primary abd secondary flow)
- `shaft_spitter`is used to manage shaft splitter (to split the shaft power into booster and fan compressor)

Aero 0D and simplified geometry are considered. 

In [ ]:
from pyturbo.systems.turbofan import Turbofan
sys = Turbofan("tf")

In [ ]:
sys.to_d3()

Turbofan is scaled from `fan_diameter`.

In [ ]:
import numpy as np

# define some init values to CFM56 order of magnitude
sys.fuel_W = 1.4
sys.fan_module.fan.geom.blade_hub_to_tip_ratio = 0.3
sys.core.compressor.geom.blade_hub_to_tip_ratio = 0.7
sys.geom.core_radius_ratio = 0.35
sys.core.compressor.aero.phiP = 0.7
sys.fan_module.booster.aero.phiP = 0.7
sys.core.turbine.aero.Ncdes = 50.
sys.fan_diameter = 1.85

# init unknowns
sys.fl_in.W = 400.
sys.fan_module.splitter_shaft.power_fractions = np.r_[0.15]
sys.fan_module.splitter_fluid.fluid_fractions = np.r_[0.15]
sys.turbine.aero.Ncqdes = 75.
sys.core.turbine.aero.Ncqdes = 75.
sys.core.turbine.sh_out.power = 45e6
sys.core.compressor.sh_in.N = 15e3
sys.turbine.sh_out.power = 30e6
sys.fan_module.sh_in.N = 4e3

sys.run_once()
sys.jupyter_view(options={
    "fan_module": dict(opacity=0.7, face_color="#92B4EC"),
    "fan_module.spinner": dict(face_color="#E1E5EA", opacity=1.),
    "fan_duct": dict(opacity=0.7),
    "core_cowl": dict(opacity=0.7),
    "nacelle": dict(face_color="#E1E5EA", opacity=0.6),
    "inlet": dict(opacity=1.),
    "gas_generator": dict(face_color="red", opacity=0.9),
    "turbine": dict(face_color="#92B4EC", opacity=0.9),
    "trf": dict(opacity=0.6),
}, view_size=(800, 600))

In [ ]:
from pyturbo.utils import add_nacelle_brand

add_nacelle_brand(sys.nacelle.geom, sys._renderer, "textures/twiinIT_logo.png")

In [ ]:
import time

total_time = 1500  # number of frames
frame_rate = 200

for i in range(total_time):
    a = 3.
    b = 6.
    theta = -0.6 * np.pi
    t = i * 2 * np.pi / total_time
    sys._renderer._cam.position = (a * np.sin(theta) * np.cos(t) + b * np.cos(theta) * np.sin(t), 1., a * np.cos(t) * np.cos(theta) - b * np.sin(theta) * np.sin(t))
    sys._renderer._cam.lookAt((0., 0., 0.))
    sys._renderer._cc.target = (0., 0., 0.)
    time.sleep(1 / frame_rate)

## direct mode
`thrust`is computed from `fuel_W`.

In [ ]:
from cosapp.drivers import NonLinearSolver
from cosapp.utils import LogLevel, set_log
from pyturbo.thermo import IdealDryAir

gas = IdealDryAir()
set_log()

In [ ]:
from cosapp.recorders import DataFrameRecorder

pamb = 101325.
tamb = 288.15 + 15.
mach = 0.25

sys.fuel_W = 1.4
sys.pamb = pamb
sys.fl_in.Tt = gas.total_t(tamb, mach)
sys.fl_in.Pt = gas.total_p(pamb, tamb, sys.fl_in.Tt)

run = sys.add_driver(NonLinearSolver('run', max_iter=50, factor=0.9, history=False))
run.add_unknown('fl_in.W')

sys.run_drivers()

print('mach ', mach)
print('pamb ', sys.pamb, 'Pa')
print('thrust ', sys.thrust * 0.224809/1e3, 'klbf')
print('N1 ', sys.N1, "rpm")
print('N2 ', sys.N2, "rpm")
print('bpr ', sys.bpr)
print('opr ', sys.opr)
print('T41 ', sys.core.turbine.fl_in.Tt, 'K')
print('sfc ', sys.sfc, 'kg/(h*kN)')

## controle mode
`fuel_W` is computed from `thrust` or `N1` value request

In [ ]:
pamb = 101325.
tamb = 288.15
mach = 0.

sys.pamb = pamb
sys.fl_in.Tt = gas.total_t(tamb, mach)
sys.fl_in.Pt = gas.total_p(pamb, tamb, sys.fl_in.Tt)

run = sys.add_driver(NonLinearSolver('run'))
run.add_unknown(['fl_in.W', 'fuel_W'])
run.add_equation('N1 == 3.5e3')
sys.run_drivers()

print('mach ', mach)
print('pamb ', sys.pamb, 'Pa')
print('thrust ', sys.thrust * 0.224809/1e3, 'klbf')
print('N1 ', sys.N1, "rpm")
print('N2 ', sys.N2, "rpm")
print('bpr ', sys.bpr)
print('opr ', sys.opr)
print('T41 ', sys.core.turbine.fl_in.Tt, 'K')
print('sfc ', sys.sfc, 'kg/(h*kN)')